In [1]:
!pip install tqdm

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
root_dir = '..'
sys.path.append(root_dir)
import utils
from LinearAlignment import LinearAlignment

/home/misha/python-evironments/py373/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/misha/python-evironments/py373/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
data = pd.read_csv(os.path.join(root_dir, 'data/train.csv'))
data['epoch'] = pd.to_datetime(data['epoch']).astype(int)

In [4]:
features_list=('x', 'y', 'z', 'Vx', 'Vy', 'Vz')

In [7]:
result_df = []
alignment_model = LinearAlignment()

satellites_list = data['sat_id'].unique()
for sat_id in tqdm(satellites_list):
    pred = pd.DataFrame([])
    sat_data = utils.get_satellite_data(data, sat_id)
    try:
        sat_data = sat_data.join(utils.remove_time_jumps(sat_data))
#         filtered_sat_data = utils.remove_time_jumps(sat_data)
    except KeyError as e:
        print(f'jump removal failed for satellite {sat_id}:\t{type(e).__name__} {e}')
        continue
#     sat_data = sat_data.join(utils.remove_time_jumps(sat_data))
    n_train = len(sat_data) // 2
    train_sat_data = sat_data[:n_train]
    
    try:
        for feature_name in features_list:
            alignment_model.fit(t=train_sat_data['epoch'].values,
                                x=train_sat_data[f'{feature_name}_sim'].values,
                                gt=train_sat_data[feature_name].values)
            pred[feature_name] = alignment_model.predict(t=sat_data['epoch'].values,
                                                         x=sat_data[f'{feature_name}_sim'].values)
    except Exception as e:
        print(f'linear alignment failed for satellite {sat_id}:\t{type(e).__name__} {e}')
        continue
        
    sat_sim_smape = utils.smape(sat_data.loc[n_train:, [f'{f}_sim' for f in features_list]].values,
                                sat_data.loc[n_train:, features_list].values)
    sat_new_smape = utils.smape(pred.loc[n_train:, features_list].values,
                                sat_data.loc[n_train:, features_list].values)
    result_df.append([sat_id, sat_sim_smape, sat_new_smape])
        

  1%|          | 6/600 [00:11<17:09,  1.73s/it]

jump removal failed for satellite 5:	KeyError 3531


 25%|██▍       | 149/600 [03:39<06:31,  1.15it/s]

jump removal failed for satellite 148:	KeyError 2275


 26%|██▌       | 157/600 [03:47<04:54,  1.51it/s]

...

 88%|████████▊ | 527/600 [13:43<01:24,  1.16s/it]

jump removal failed for satellite 526:	KeyError 3130


 92%|█████████▏| 554/600 [14:32<00:47,  1.03s/it]

jump removal failed for satellite 553:	KeyError 4525


100%|██████████| 600/600 [15:50<00:00,  1.58s/it]


In [8]:
result_df = pd.DataFrame(result_df, columns=['sat_id', 'sat_simulation_smape', 'sat_new_smape'])

In [9]:
print(f'{len(result_df)}/{len(satellites_list)} satellites processed')
print('\nmetrics:', dict(result_df.iloc[:, 1:].mean()))

587/600 satellites processed

metrics: {'sat_simulation_smape': 0.2132816584458697, 'sat_new_smape': 0.12919047242986784}
